In [ ]:
cd /content/drive/MyDrive/UMass/"685 PROJECT"/"LSR-VQ"

/content/drive/MyDrive/UMass/685 PROJECT/LSR-VQ


In [ ]:
# Install required libraries
!pip install torch ir_datasets wandb numpy scikit-learn sentence-transformers transformers tqdm scipy matplotlib rank-eval
!pip install faiss-cpu
# !pip uninstall faiss-gpu-cu11

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import libraries
import pandas as pd
import csv

import torch
import ir_datasets
import faiss
import wandb
import heapq
import time
import sys
import random
import string
import os
import pickle
import math

import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
from sklearn.metrics import ndcg_score, recall_score
from collections import defaultdict
from scipy.sparse import csr_matrix
from collections import defaultdict
from tqdm import tqdm
from rank_eval import Qrels, Run, evaluate

## Download Dataset (Don't run!!! Data is already downloaded)

In [ ]:
# # Download collection (pId -> passage text)
# !wget -P data/raw/ https://msmarco.z22.web.core.windows.net/msmarcoranking/collection.tar.gz

# # Download queries (qId -> query text)
# !wget -P data/raw/ https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz

# # Download qRels Dev and Train
# !wget -P data/raw/ https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.dev.tsv
# !wget -P data/raw/ https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.train.tsv

# # Download training data - qId positive_pId and negative_pId
# !wget -P data/raw/ https://msmarco.z22.web.core.windows.net/msmarcoranking/qidpidtriples.train.full.2.tsv.gz

# !wget -P data/raw https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz

In [ ]:
# # unzip files
# !tar -xf "data/raw/queries.tar.gz"
# !tar -xf "data/raw/collection.tar.gz"
# !tar -xf "data/raw/qidpidtriples.train.full.2.tsv.gz"
# !tar -xf "data/raw/collectionandqueries.tar.gz"

## Dataset

In [ ]:
# Read the tsv file as a dictionary
def open_file(file_path, keys = [0, 1]):
    with open(file_path, mode = "r", encoding = "utf-8") as file:
        reader = csv.reader(file, delimiter= "\t")

        data = {}
        for row in reader:
            if row[keys[0]] in data:
                data[row[keys[0]]].append(row[keys[1]])
            else:
                data[row[keys[0]]] = [row[keys[1]]]

    return data

# Load and preprocess the dataset
def load_and_preprocess_dataset():
    # Load all passages
    passages = open_file("data/collection.tsv")

    # Load all train, dev & eval queries
    queries_train = open_file("data/queries.train.tsv")
    queries_dev = open_file("data/queries.dev.small.tsv")
    # queries_eval = open_file("data/queries.eval.tsv")

    # Load qRels train & dev
    # These only have relevant files i.e binary relevance judgment score
    # will be 1 for all the entries
    qrels_train = open_file("data/qrels.train.tsv", keys = [0, 2])
    qrels_dev = open_file("data/qrels.dev.small.tsv", keys = [0, 2])

    return passages, queries_train, queries_dev, qrels_train, qrels_dev

In [ ]:
passages, queries_train, queries_dev, qrels_train, qrels_dev = load_and_preprocess_dataset()

In [ ]:
def print_samples(file_name, dict_, n = 2):
    print('-' * 15)
    print(f'Statistics for {file_name}:')
    print('Total number of samples:', len(dict_))
    samples = list(dict_.items())[:2]
    for entry in samples:
        print(entry)

print_samples("passages", passages)
print_samples("queries_train", queries_train)
print_samples("queries_dev", queries_dev)
print_samples("qrels_train", qrels_train)
print_samples("qrels_dev", qrels_dev)

---------------
Statistics for passages:
Total number of samples: 8841823
('0', ['The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'])
('1', ['The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.'])
---------------
Statistics for queries_train:
Total number of samples: 808731
('121352', ['define extreme'])
('634306', ['what does chattel mean on credit history'])
---------------
Statistics for queries_dev:
Total number of samples: 6980
('1048585', ["what is paula deen's brother"])
('2', [' Androgen receptor define'])
---------------
Statistics for qrels_train:
Total number of samples: 502939
('1

## Initialize Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained('facebook/contriever-msmarco')
model = AutoModel.from_pretrained('facebook/contriever-msmarco').to(device)

# Mean pooling
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
print('Using device', device)

Using device cuda


## Evaluate Model

In [ ]:
# Evaluate model
def evaluate_model(model, passages, dev_queries, dev_qrels, tokenizer, device, batch_size_inference = 128, min_weight = 1e-5, query_topk = 128, doc_topk = 1024, quick_mode = True):
    base_model = model.module if isinstance(model, nn.DataParallel) else model

    model.eval()
    all_results = {}  # Store results for each query

    # Filter passages if in quick mode
    if quick_mode:
        relevant_passage_ids = set()
        for qid in dev_qrels:
            relevant_passage_ids.update(dev_qrels[qid])
        passages = {passage_id: passages[passage_id] for passage_id in relevant_passage_ids}
        print(f"Quick mode: using {len(passages)} passages for evaluation")

    # Filter queries if in quick mode
    if quick_mode:
        dev_queries = dict(list(dev_queries.items()))
        dev_qrels = {qid: dev_qrels[qid][0] for qid in dev_queries if qid in dev_qrels}
        print(f"Quick mode: using {len(dev_queries)} queries for evaluation")

    # Dense retrieval using FAISS
    print("Building FAISS index...")
    passage_ids = list(passages.keys())
    passage_embeddings = []

    # Encode all passages
    for i in tqdm(range(0, len(passage_ids), batch_size_inference), desc = "Encoding passages"):
        batch_passages = [passages[pid][0] for pid in passage_ids[i:i + batch_size_inference]]

        # Pad till the model's configured max_len (512)
        passage_inputs = tokenizer(batch_passages, padding = True, truncation = True, return_tensors = 'pt')
        passage_inputs = {k: v.to(device) for k, v in passage_inputs.items()}

        with torch.no_grad():
            outputs = model(passage_inputs["input_ids"], passage_inputs["attention_mask"])
            batch_embeddings = mean_pooling(outputs[0], passage_inputs['attention_mask'])
            passage_embeddings.append(batch_embeddings.cpu().numpy())

    passage_embeddings = np.vstack(passage_embeddings)

    # Build FAISS index
    dimension = passage_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(passage_embeddings)

    # Evaluate queries
    with torch.no_grad():
        for qid, query in tqdm(dev_queries.items(), desc = "Evaluating"):
            if qid not in dev_qrels:
                continue

            query = query[0]
            # Pad till the model's configured max_len (512)
            query_input = tokenizer(query, padding = True, truncation = True, return_tensors = 'pt')
            query_input = {k: v.to(device) for k, v in query_input.items()}
            output = model(query_input["input_ids"], query_input["attention_mask"])
            query_embedding = mean_pooling(output[0], query_input['attention_mask'])
            query_embedding = query_embedding.cpu().numpy()

            # Search using FAISS
            scores, indices = index.search(query_embedding, 1000)
            search_results = [(passage_ids[idx], float(score)) for idx, score in zip(indices[0], scores[0])]
            all_results[qid] = search_results

    # Create rank_eval Run and Qrels objects
    run = Run()
    qrels = Qrels()

    # Add results to Run object
    for qid in all_results:
        passage_ids = [str(passage_id) for passage_id, score in all_results[qid]]
        scores = [float(score) for _, score in all_results[qid]]
        run.add(qid, passage_ids, scores)

    # Add relevance judgments to Qrels object
    for qid in dev_qrels:
        qrels.add(qid, [str(doc_id) for doc_id in dev_qrels[qid]], [1] * len(dev_qrels[qid]))

    # Evaluate using rank_eval
    metrics = ["ndcg@10", "ndcg@100", "ndcg@1000", "recall@10", "recall@100", "recall@1000", "mrr@10"]
    results = evaluate(qrels, run, metrics)

    return (
        results["mrr@10"],
        {
            '10': results["ndcg@10"],
            '100': results["ndcg@100"],
            '1000': results["ndcg@1000"]
        },
        {
            '10': results["recall@10"],
            '100': results["recall@100"],
            '1000': results["recall@1000"]
        }
    )


In [ ]:
mrr_10, ndcg, recall = evaluate_model(model = model, passages = passages, dev_queries = queries_dev, dev_qrels = qrels_dev, tokenizer = tokenizer, device = device)

Quick mode: using 7433 passages for evaluation
Quick mode: using 6980 queries for evaluation
Building FAISS index...


Evaluating: 100%|██████████| 6980/6980 [01:18<00:00, 89.45it/s]


KeyboardInterrupt: 

In [ ]:
# Metrics
print("Final Evaluation")
print(f"MRR@10: {mrr_10:.4f}")
print(f"nDCG@10: {ndcg['10']:.4f}, nDCG@100: {ndcg['100']:.4f}, nDCG@1000: {ndcg['1000']:.4f}")
print(f"Recall@10: {recall['10']:.4f}, Recall@100: {recall['100']:.4f}, Recall@1000: {recall['1000']:.4f}")

Final Evaluation
MRR@10: 0.6667
nDCG@10: 0.7468, nDCG@100: 0.7468, nDCG@1000: 0.7468
Recall@10: 1.0000, Recall@100: 1.0000, Recall@1000: 1.0000


## Get query/document embeddings (In Progress)

In [ ]:
# Split the embedding to k chunks
def split_embedding_into_chunks(embeddings, k_chunks):
    bsz, dim = embeddings.shape
    assert dim % k_chunks == 0
    return embeddings.view(bsz * k_chunks, dim // k_chunks)


def get_embeddings(model, passages, queries, qrels, tokenizer, device, batch_size_inference = 128, min_weight = 1e-5, query_topk = 128, doc_topk = 1024, quick_mode = True, output_batching = False):
    base_model = model.module if isinstance(model, nn.DataParallel) else model

    # Filter passages if in quick mode
    if quick_mode:
        relevant_passage_ids = set()
        for qid in qrels:
            relevant_passage_ids.update(qrels[qid])
        print(len(relevant_passage_ids))
        # passages = {passage_id: passages[passage_id] for passage_id in relevant_passage_ids}
        print(f"Quick mode: using {len(passages)} passages for evaluation")
        return

    # Filter queries if in quick mode
    if quick_mode:
        queries = dict(list(queries.items())[:5000])
        qrels = {qid: qrels[qid] for qid in queries if qid in qrels}
        print(f"Quick mode: using {len(queries)} queries for evaluation")

    passage_ids = list(passages.keys())
    passage_embeddings = []

    # Encode all passages
    for i in tqdm(range(0, len(passage_ids), batch_size_inference), desc = "Encoding passages"):
        batch_passages = [passages[pid] for pid in passage_ids[i:i + batch_size_inference]]

        # Pad till the model's configured max_len (512)
        passage_inputs = tokenizer(batch_passages, padding = True, truncation = True, return_tensors = 'pt')
        passage_inputs = {k: v.to(device) for k, v in passage_inputs.items()}

        with torch.no_grad():
            outputs = model(passage_inputs["input_ids"], passage_inputs["attention_mask"])
            batch_embeddings = mean_pooling(outputs[0], passage_inputs['attention_mask'])
            batch_chunks = split_embedding_into_chunks(batch_embeddings, k_chunks = 4)
            passage_embeddings.append(batch_chunks)


    query_ids = list(qrels.keys())
    query_embeddings = []

    # Encode all queries
    for i in tqdm(range(0, len(query_ids), batch_size_inference), desc = "Encoding queries"):
        batch_queries = [queries[pid] for pid in query_ids[i:i + batch_size_inference]]

        # Pad till the model's configured max_len (512)
        query_inputs = tokenizer(batch_queries, padding = True, truncation = True, return_tensors = 'pt')
        query_inputs = {k: v.to(device) for k, v in query_inputs.items()}

        with torch.no_grad():
            outputs = model(query_inputs["input_ids"], query_inputs["attention_mask"])
            batch_embeddings = mean_pooling(outputs[0], query_inputs['attention_mask'])
            batch_chunks = split_embedding_into_chunks(batch_embeddings, k_chunks = 4)
            query_embeddings.append(batch_chunks)

    if output_batching == True:
        passage_embeddings = torch.cat(passage_embeddings, dim = 0)
        query_embeddings = torch.cat(query_embeddings, dim = 0)

    return passage_embeddings, query_embeddings

In [ ]:
passage_embeddings, query_embeddings = get_embeddings(model, passages, queries_dev, qrels_dev, tokenizer = tokenizer, device = device)

10
Quick mode: using 8841823 passages for evaluation
passage Ids Buffered data was truncated after reaching the output size limit.

## Vector Quantizer (In Progress)

In [ ]:
class Quantize(nn.Module):
    def __init__(self, dim, n_embed, decay = 0.99, eps = 1e-5):
        super().__init__()

        self.dim = dim
        self.n_embed = n_embed
        self.decay = decay
        self.eps = eps

        embed = torch.randn(dim, n_embed)
        self.register_buffer("embed", embed)
        self.register_buffer("cluster_size", torch.zeros(n_embed))
        self.register_buffer("embed_avg", embed.clone())

    def forward(self, input):
        flatten = input.reshape(-1, self.dim)
        dist = (
            flatten.pow(2).sum(1, keepdim = True)
            - 2 * flatten @ self.embed
            + self.embed.pow(2).sum(0, keepdim = True)
        )
        _, embed_ind = (-dist).max(1)
        embed_onehot = F.one_hot(embed_ind, self.n_embed).type(flatten.dtype)
        embed_ind = embed_ind.view(*input.shape[:-1])
        quantize = self.embed_code(embed_ind)

        if self.training:
            embed_onehot_sum = embed_onehot.sum(0)
            embed_sum = flatten.transpose(0, 1) @ embed_onehot

            self.cluster_size.data.mul_(self.decay).add_(
                embed_onehot_sum, alpha=1 - self.decay
            )
            self.embed_avg.data.mul_(self.decay).add_(embed_sum, alpha=1 - self.decay)
            n = self.cluster_size.sum()
            cluster_size = (
                (self.cluster_size + self.eps) / (n + self.n_embed * self.eps) * n
            )
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(0)
            self.embed.data.copy_(embed_normalized)

        quantize = input + (quantize - input).detach()

        return quantize, embed_ind

    def embed_code(self, embed_id):
        return F.embedding(embed_id, self.embed.transpose(0, 1))

## Train/Inference VQ (In Progress)

In [ ]:
def train_VQ(model, passages, queries, train_qrels, dev_queries, dev_qrels, tokenizer, device, eval_only = False):
    """
    Function to run the training

    Args:
        eval_only: If True, only run evaluation using a pre-trained model
    """

    # Model configuration
    freeze_bert = False
    num_epochs = 100
    query_topk = 128  # Top-k dimensions to keep in query embeddings for retrieval
    doc_topk = 512  # Top-k dimensions to keep in document embeddings for retrieval
    evaluation_steps = 16_000
    batch_size = 32
    batch_size_inference = 64

    passage_embeddings, query_embeddings = get_embeddings(model, passages, queries, train_qrels, tokenizer = tokenizer, device = device, output_batching = True)
    embeddings = torch.cat((query_embeddings, passage_embeddings), dim = 0)

    # Train the codebook vectors
    quantizer = Quantize(dim = 192, n_embed = 32)
    quantizer.training = True
    for emb in embeddings:
        _, code = quantizer(emb.unsqueeze(0))

    return quantizer


def inference_VQ(model, quantizer, passages, queries, train_qrels, dev_queries, dev_qrels, tokenizer, device, eval_only = False):
    """
    Function to run the inference on VQ
    """

    # Get the embeddings for the dev set
    passage_embeddings, query_embeddings = get_embeddings(model, passages, dev_queries, dev_qrels, tokenizer = tokenizer, device = device, output_batching = False)

    # Get the code book vectors for each passage and build the inverted index
    # passage_embeddings - list of chunked embeddings(for each example)
    quantizer.training = False
    for emb in passage_embeddings:
        # emb - (num_chunks, dim)
        _, code = quantizer(emb)
        code = code.view(-1, 4)
        print('Code', code)

    return quantizer

In [ ]:
# passages, queries_train, queries_dev, queries_eval, qrels_train, qrels_dev
quantizer = train_VQ(model, passages, queries_train, qrels_train, queries_dev, qrels_dev, tokenizer, device)

NameError: name 'train_VQ' is not defined

In [ ]:
inference_VQ(model, quantizer, passages, queries_train, qrels_train, queries_dev, qrels_dev, tokenizer, device)

Buffered data was truncated after reaching the output size limit.